In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import operator
from infomap import infomap
from pymongo import MongoClient
%matplotlib inline

In [2]:
filename = "/Users/arefindk/Desktop/github_merged_pull/follow.graphml"
G = nx.read_graphml(filename)

In [3]:
print "Nodes ", G.number_of_nodes()
print "Edges ", G.number_of_edges()

Nodes  771078
Edges  2469419


In [4]:
## Load mongodb
client = MongoClient("mongodb://localhost:27017/")
db = client.github_follow
users = db.users

In [5]:
## Some lists
removedNodes = list()
allNodes = G.nodes()
numNodes = len(allNodes)
count = 0
## Getting language attribute of the user
for currentNode in allNodes:
    ## Finding a user with specific login name from the network
    findDoc = {"login":currentNode}
    result = users.find_one(findDoc)

    if not result:
        removedNodes.append(currentNode)
    else:
        nativeLanguage = result['native_language']

        if nativeLanguage== None:
            removedNodes.append(currentNode)
        else:
            G.node[currentNode]['lang'] = str(nativeLanguage)
            G.node[currentNode]['public_repos'] = int(result['public_repos'])
            f = "%d/%m/%Y %H:%M:%S"
            G.node[currentNode]['created_at'] = result['created_at'].strftime(f)
#             if not result['company'] == None:
#                 G.node[currentNode]['company'] = str(result['company'])
#             else:
#                 G.node[currentNode]['company'] = ""
#             if not result['location'] == None:
#                 G.node[currentNode]['location'] = str(result['location'])
#             else:
#                 G.node[currentNode]['location'] = ""
            G.node[currentNode]['type'] = str(result['type'])
            G.node[currentNode]['latest_followers'] = int(result['followers'])
            G.node[currentNode]['latest_followings'] = int(result['following'])
    count += 1
    if count % 10000 == 0:
        print "Remaining ", numNodes - count

Remaining  761078
Remaining  751078
Remaining  741078
Remaining  731078
Remaining  721078
Remaining  711078
Remaining  701078
Remaining  691078
Remaining  681078
Remaining  671078
Remaining  661078
Remaining  651078
Remaining  641078
Remaining  631078
Remaining  621078
Remaining  611078
Remaining  601078
Remaining  591078
Remaining  581078
Remaining  571078
Remaining  561078
Remaining  551078
Remaining  541078
Remaining  531078
Remaining  521078
Remaining  511078
Remaining  501078
Remaining  491078
Remaining  481078
Remaining  471078
Remaining  461078
Remaining  451078
Remaining  441078
Remaining  431078
Remaining  421078
Remaining  411078
Remaining  401078
Remaining  391078
Remaining  381078
Remaining  371078
Remaining  361078
Remaining  351078
Remaining  341078
Remaining  331078
Remaining  321078
Remaining  311078
Remaining  301078
Remaining  291078
Remaining  281078
Remaining  271078
Remaining  261078
Remaining  251078
Remaining  241078
Remaining  231078
Remaining  221078
Remaining 

In [6]:
print len(removedNodes)

226439


In [7]:
all_nodes_set = set(allNodes)

In [8]:
removed_nodes_set = set(removedNodes)

In [9]:
trimmed_node_set = all_nodes_set - removed_nodes_set

In [10]:
len(trimmed_node_set)

544639

In [11]:
trimmed_subgraph = G.subgraph(trimmed_node_set)

In [12]:
print trimmed_subgraph.number_of_nodes()
print trimmed_subgraph.number_of_edges()

544639
1835281


In [13]:
langs = nx.get_node_attributes(trimmed_subgraph,'lang')
lang_set = set(langs.values())
print len(lang_set), lang_set

239 set(['Mercury', 'Mako', 'TypeScript', 'DCPU-16 ASM', 'PureBasic', 'Objective-C++', 'DTrace', 'Delphi', 'Lean', 'NewLisp', 'Jupyter Notebook', 'Rebol', 'eC', 'Frege', 'Groff', 'AspectJ', 'Brightscript', 'Shell', 'Web Ontology Language', 'AppleScript', 'Eiffel', 'Nix', 'SuperCollider', 'Nginx', 'MTML', 'Opa', 'Elixir', 'SAS', 'Agda', 'Logos', 'D', 'Kotlin', 'J', 'Standard ML', 'Objective-C', 'Batchfile', 'Bison', 'Oz', 'Objective-J', 'NetLinx', 'LoomScript', 'Handlebars', 'Gosu', 'Swift', 'Nemerle', 'Ceylon', 'Smarty', 'Pure Data', 'Arc', 'Brainfuck', 'IGOR Pro', 'APL', 'SQL', 'Go', 'Visual Basic', 'PHP', 'SQF', 'Glyph', 'Coq', 'Java', 'Scala', 'OpenSCAD', 'ApacheConf', 'Makefile', 'BitBake', 'Perl', 'Elm', 'Lua', 'Vue', 'GDScript', 'Verilog', 'Stata', 'Factor', 'Haxe', 'Forth', 'Red', 'COBOL', 'XProc', 'XPages', 'LSL', 'RobotFramework', 'CoffeeScript', 'HTML', 'Awk', 'Lex', 'UnrealScript', 'Idris', 'Ruby', 'LilyPond', 'PLpgSQL', 'Self', 'C', 'AutoHotkey', 'Isabelle', 'VCL', 'Hack', 

In [14]:
user_type = nx.get_node_attributes(trimmed_subgraph,'type')

In [15]:
same_type_of_users = {}
for k,v in user_type.iteritems():
    same_type_of_users.setdefault(v,[]).append(k)

In [16]:
print same_type_of_users.keys()

['Organization', 'User']


In [17]:
users_organization = same_type_of_users['Organization']
print len(users_organization)

9374


In [18]:
users_user = same_type_of_users['User']
print len(users_user)

535265


In [19]:
trimmed_subgraph_users = G.subgraph(users_user)
trimmed_subgraph_organizations = G.subgraph(users_organization)

In [20]:
print trimmed_subgraph_users.number_of_edges()

1799964


In [21]:
print trimmed_subgraph_organizations.number_of_edges()

75


In [22]:
components_G = nx.strongly_connected_components(trimmed_subgraph_users)
components_G = sorted(components_G, key = lambda c:len(c), reverse = True)
largest_strongly_connected_component = components_G[0]
induced_strongly_connected_subgraph = trimmed_subgraph_users.subgraph(largest_strongly_connected_component)
print induced_strongly_connected_subgraph.number_of_edges()
## Now converting it from a Multi_Di_Graph to a simple Di_Graph
induced_strongly_connected_subgraph = nx.DiGraph(induced_strongly_connected_subgraph)
print induced_strongly_connected_subgraph.number_of_edges()

1005583
1005583


In [23]:
induced_strongly_connected_subgraph_int = nx.convert_node_labels_to_integers(induced_strongly_connected_subgraph, first_label = 0, ordering = 'default', label_attribute = "login")

In [24]:
print len(largest_strongly_connected_component)
print induced_strongly_connected_subgraph_int.number_of_nodes(), induced_strongly_connected_subgraph_int.number_of_edges()

154859
154859 1005583


In [25]:
nx.write_graphml(induced_strongly_connected_subgraph_int, "trimmed_follow_users_strong_int.graphml")